In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00


In [3]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 76.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 101.4 MB/s eta 0:00:00


In [4]:
import numpy as np
import pandas as pd
import re
import tensorflow_datasets as tfds
#to avoid doing pre-processing multiple times and thus saving RAM for further tasks
import pickle
import nltk
from nltk.stem import WordNetLemmatizer
import torch
from torch import nn 
from datasets import load_dataset
from transformers import PegasusForConditionalGeneration, PegasusTokenizer, Trainer, TrainingArguments

In [5]:
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

## **Using popular Cnn DailyMail Dataset**

In [ ]:
dataset = tfds.load('cnn_dailymail', split='train')

## **Checking the format of the dataset**

In [ ]:
dataset

<_PrefetchDataset element_spec={'article': TensorSpec(shape=(), dtype=tf.string, name=None), 'highlights': TensorSpec(shape=(), dtype=tf.string, name=None), 'id': TensorSpec(shape=(), dtype=tf.string, name=None), 'publisher': TensorSpec(shape=(), dtype=tf.string, name=None)}>

In [ ]:
print(len(dataset))

287113


## **Converting dataset to Dataframe for performing pre-processing efficiently**

In [ ]:
df = pd.DataFrame(dataset)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287113 entries, 0 to 287112
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   article     287113 non-null  object
 1   highlights  287113 non-null  object
 2   id          287113 non-null  object
 3   publisher   287113 non-null  object
dtypes: object(4)
memory usage: 8.8+ MB


## **Considering only first 2000 rows of the dataset**

In [ ]:
datas = df[0:2000]
datas.info

<bound method DataFrame.info of                                                 article  \
0     tf.Tensor(b"By. Associated Press. PUBLISHED:. ...   
1     tf.Tensor(b'(CNN) -- Ralph Mata was an interna...   
2     tf.Tensor(b"A drunk driver who killed a young ...   
3     tf.Tensor(b"(CNN) -- With a breezy sweep of hi...   
4     tf.Tensor(b"Fleetwood are the only team still ...   
...                                                 ...   
1995  tf.Tensor(b"Firemen in Germany were forced to ...   
1996  tf.Tensor(b'The 298 victims on board tragic fl...   
1997  tf.Tensor(b"By. Charlie Skillen. Follow @@char...   
1998  tf.Tensor(b"By. Martin Robinson. PUBLISHED:. 0...   
1999  tf.Tensor(b'By. Nick Harris. What price a Worl...   

                                             highlights  \
0     tf.Tensor(b'Bishop John Folda, of North Dakota...   
1     tf.Tensor(b'Criminal complaint: Cop used his r...   
2     tf.Tensor(b"Craig Eccleston-Todd, 27, had drun...   
3     tf.Tensor(b"Nina 

In [ ]:
datas

,article,highlights,id,publisher
0,"tf.Tensor(b""By. Associated Press. PUBLISHED:. ...","tf.Tensor(b'Bishop John Folda, of North Dakota...",tf.Tensor(b'160e9e1c25c81788df73b06b0ce955897f...,"tf.Tensor(b'dm', shape=(), dtype=string)"
1,tf.Tensor(b'(CNN) -- Ralph Mata was an interna...,tf.Tensor(b'Criminal complaint: Cop used his r...,tf.Tensor(b'3b07f5102c69e3e609d73b2ccb0dc5549d...,"tf.Tensor(b'cnn', shape=(), dtype=string)"
2,"tf.Tensor(b""A drunk driver who killed a young ...","tf.Tensor(b""Craig Eccleston-Todd, 27, had drun...",tf.Tensor(b'2fd252e716c21fd523becd24774bdd6220...,"tf.Tensor(b'dm', shape=(), dtype=string)"
3,"tf.Tensor(b""(CNN) -- With a breezy sweep of hi...","tf.Tensor(b""Nina dos Santos says Europe must b...",tf.Tensor(b'6e3c654d92c1f34c2355a2a392fb5ef618...,"tf.Tensor(b'cnn', shape=(), dtype=string)"
4,"tf.Tensor(b""Fleetwood are the only team still ...",tf.Tensor(b'Fleetwood top of League One after ...,tf.Tensor(b'df5e0d63e3cd78096cdbf073232a3aa5f6...,"tf.Tensor(b'dm', shape=(), dtype=string)"
...,...,...,...,...
1995,"tf.Tensor(b""Firemen in Germany were forced to ...","tf.Tensor(b""Conditions inside the flat in Nurt...",tf.Tensor(b'7dbbca7347243805270200bf832dac247b...,"tf.Tensor(b'dm', shape=(), dtype=string)"
1996,tf.Tensor(b'The 298 victims on board tragic fl...,"tf.Tensor(b""Expert says MH17 passengers would ...",tf.Tensor(b'13478c1a65ab94706c742b225c5e520333...,"tf.Tensor(b'dm', shape=(), dtype=string)"
1997,"tf.Tensor(b""By. Charlie Skillen. Follow @@char...","tf.Tensor(b""Diego Costa took part in Atletico ...",tf.Tensor(b'6be43033f88fe75d17ac48ad775bd70b98...,"tf.Tensor(b'dm', shape=(), dtype=string)"
1998,"tf.Tensor(b""By. Martin Robinson. PUBLISHED:. 0...","tf.Tensor(b""Matt Pullen was due to become Mana...",tf.Tensor(b'1521c8280248f2ae39942306c50f860f37...,"tf.Tensor(b'dm', shape=(), dtype=string)"


## **Pre-processing begins:**

In [ ]:
#Dropping the unnecessary columns from the dataset
datas = datas.drop(['id','publisher'], axis=1)

In [ ]:
#Selecting the unnecesaary words
words_to_remove = ['tf', 'Tensor', 'shape','numpy','b', '--', ',']

In [ ]:
#Removing unnecessary words from the every row
remove_words = lambda x: ' '.join([word for word in x.numpy().decode().split() if word.lower() not in words_to_remove])

In [ ]:
#Removing selected words from column highlights
datas['highlights'] = datas['highlights'].apply(remove_words)

In [ ]:
#Removing selected words from column article
datas['article'] = datas['article'].apply(remove_words)

In [ ]:
#Renaming the column from highlights to summary for more relatibilty
datas.rename(columns = {'highlights':'Summary'}, inplace = True)

In [ ]:
#Renaming the column from article to Article for more relatibilty
datas.rename(columns = {'article':'Article'}, inplace = True)

In [ ]:
#Converting and downloading the dataset to csv preventing it from downloading it everytime from tensordataflow
download = datas.to_csv('Dataset.csv')

In [ ]:
main_df = datas

In [ ]:
# pattern to match non-alphanumeric and non-space characters
def remove_special_chars(text):
    pattern = r'[^a-zA-z0-9\s]' 
    return re.sub(pattern, '', text)

In [ ]:
#Cleaning the dataset by removing the special characters from the dataset
main_df = main_df.applymap(remove_special_chars)

In [ ]:
#Converting every row to lowercase
main_df = main_df.applymap(lambda x: x.lower())

In [ ]:
#Removing duplicate values if exist but in the dataset there are 0 duplicates
main_df.duplicated(subset= ['Article', 'Summary']).sum()

0

In [ ]:
main_df = main_df.drop_duplicates(subset= ['Article', 'Summary'])
main_df.shape

(2000, 2)

In [ ]:
#Initialising the contractions for word expansion
contractions = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are"
}

In [ ]:
# Function to expand contractions
def expand_contractions(text):
    for contraction, expansion in contractions.items():
        text = text.replace(contraction, expansion)
    return text

In [ ]:
# applying contractions and expansion functions to column Article
main_df['Article'] = main_df['Article'].apply(expand_contractions)

In [ ]:
# applying contractions and expansion functions to column Summary
main_df['Summary'] = main_df['Summary'].apply(expand_contractions)

In [ ]:
#Function to remove all the URLs 
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub('', text)

In [ ]:
# applying removal URLs function to Article column
main_df['Article'] = main_df['Article'].apply(remove_urls)

In [ ]:
# applying removal URLs function to Summary column
main_df['Summary'] = main_df['Summary'].apply(remove_urls)

In [ ]:
#Generating pickle file to avoid pre-processing on every session
filename = 'train.pkl'

In [ ]:
#Run this cell only one time... and from the next time comment this cell and only import pickle and upload the pkl file, no need to run above cells
#Creating the pickle file
pickle.dump(main_df,open(filename,'wb'))

In [7]:
#For loading the saved pre-processed data from pickle file
main_df = pickle.load(open('/content/drive/MyDrive/train.pkl','rb'))

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## **Lemmatization** : 

**Doing the lemmatization but as we require the abstractive summary of the article, doing lemmatization may reduce the semantic accuracy of the article. 
I planned to trained the model with or without lemmatization to observe the performance of the model**

In [8]:
# initializing lemmatizer
lemmatizer = WordNetLemmatizer()

In [9]:
#Function of lemmatizing the every word of every row
def lemmatize_text(text):
    if not isinstance(text, str):
        return ''
    # tokenize text into words
    words = nltk.word_tokenize(text)
    # lemmatize each word and join back into a string
    return ' '.join([lemmatizer.lemmatize(word) for word in words if word.isalpha()])

In [10]:
# applying function to column Article
main_df['Article'] = main_df['Article'].apply(lemmatize_text)

In [11]:
#applying function to column Summary
main_df['Summary'] = main_df['Summary'].apply(lemmatize_text)

## **POS Tagging:**

**POS Tagging helps model to generate the summary which is more semantically as well as syntatically correct.**

In [12]:
# define function to perform POS tagging
def pos_tag_text(text):
    # tokenize text into words
    words = nltk.word_tokenize(text)
    # perform POS tagging on words
    tagged_words = nltk.pos_tag(words)
    # return list of tagged words
    return tagged_words

In [13]:
# applying function to Article only as model will use it also,Summary column does not require it
main_df['pos_tags'] = main_df['Article'].apply(pos_tag_text)

In [16]:
main_df['pos_tags'][0]

[('by', 'IN'),
 ('associated', 'JJ'),
 ('press', 'NN'),
 ('published', 'VBN'),
 ('est', 'JJS'),
 ('october', 'NN'),
 ('updated', 'JJ'),
 ('est', 'JJS'),
 ('october', 'IN'),
 ('the', 'DT'),
 ('bishop', 'NN'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('fargo', 'NN'),
 ('catholic', 'JJ'),
 ('diocese', 'NN'),
 ('in', 'IN'),
 ('north', 'JJ'),
 ('dakota', 'JJ'),
 ('ha', 'NN'),
 ('exposed', 'VBN'),
 ('potentially', 'RB'),
 ('hundred', 'CD'),
 ('of', 'IN'),
 ('church', 'NN'),
 ('member', 'NN'),
 ('in', 'IN'),
 ('fargo', 'JJ'),
 ('grand', 'JJ'),
 ('fork', 'NN'),
 ('and', 'CC'),
 ('jamestown', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('hepatitis', 'NN'),
 ('a', 'DT'),
 ('virus', 'NN'),
 ('in', 'IN'),
 ('late', 'JJ'),
 ('september', 'NN'),
 ('and', 'CC'),
 ('early', 'JJ'),
 ('october', 'NN'),
 ('the', 'DT'),
 ('state', 'NN'),
 ('health', 'NN'),
 ('department', 'NN'),
 ('ha', 'NN'),
 ('issued', 'VBD'),
 ('an', 'DT'),
 ('advisory', 'NN'),
 ('of', 'IN'),
 ('exposure', 'NN'),
 ('for', 'IN'),
 ('anyone', 'NN'),


## **Named Entity Relationship**

**Doing NER which will assign the name to every word.** 

In [14]:
# define function to perform NER
def ner_text(text):
    # tokenize text into sentences
    sentences = nltk.sent_tokenize(text)
    # tokenize each sentence into words and perform POS tagging
    sentences = [nltk.word_tokenize(sent) for sent in sentences]
    sentences = [nltk.pos_tag(sent) for sent in sentences]
    # perform NER on tagged sentences
    named_entities = []
    for tagged_sentence in sentences:
        named_entities.append(nltk.ne_chunk(tagged_sentence, binary=True))
    # return list of named entities
    return named_entities

In [15]:
# applying function to column Article only as Summary does not require it
main_df['named_entities'] = main_df['Article'].apply(ner_text)

In [15]:
main_df['named_entities'][0]

[Tree('S', [('by', 'IN'), ('associated', 'JJ'), ('press', 'NN'), ('published', 'VBN'), ('est', 'JJS'), ('october', 'NN'), ('updated', 'JJ'), ('est', 'JJS'), ('october', 'IN'), ('the', 'DT'), ('bishop', 'NN'), ('of', 'IN'), ('the', 'DT'), ('fargo', 'NN'), ('catholic', 'JJ'), ('diocese', 'NN'), ('in', 'IN'), ('north', 'JJ'), ('dakota', 'JJ'), ('ha', 'NN'), ('exposed', 'VBN'), ('potentially', 'RB'), ('hundred', 'CD'), ('of', 'IN'), ('church', 'NN'), ('member', 'NN'), ('in', 'IN'), ('fargo', 'JJ'), ('grand', 'JJ'), ('fork', 'NN'), ('and', 'CC'), ('jamestown', 'NN'), ('to', 'TO'), ('the', 'DT'), ('hepatitis', 'NN'), ('a', 'DT'), ('virus', 'NN'), ('in', 'IN'), ('late', 'JJ'), ('september', 'NN'), ('and', 'CC'), ('early', 'JJ'), ('october', 'NN'), ('the', 'DT'), ('state', 'NN'), ('health', 'NN'), ('department', 'NN'), ('ha', 'NN'), ('issued', 'VBD'), ('an', 'DT'), ('advisory', 'NN'), ('of', 'IN'), ('exposure', 'NN'), ('for', 'IN'), ('anyone', 'NN'), ('who', 'WP'), ('attended', 'VBD'), ('five'

In [ ]:
#Generating pickle file for saving the further semantic analysis done on dataset
filename1= "training.pkl"
filename2 = "valid.pkl"
filename3 = "test.pkl"

In [42]:
#Spiltting the dataset in training, validation and test dataset to train our pre-trained model
#Training on first 50 rows as of now
train_df1 = main_df['Article'][0:50]
valid_df1 = main_df['Summary'][0:50]                   
test_df1 = main_df.loc[51:61, ["Article", "Summary"]]

In [ ]:
#Storing the train,validate and test dataset to csv for further easy access
train_df1.to_csv('train_df1.csv', index=False)
valid_df1.to_csv('valid_df1.csv', index=False)
test_df1.to_csv('test_df1.csv', index=False)

In [ ]:
train_df1.to_frame().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  50 non-null     object
dtypes: object(1)
memory usage: 800.0+ bytes


In [ ]:
valid_df1.to_frame().info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50 entries, 0 to 49
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Summary  50 non-null     object
dtypes: object(1)
memory usage: 800.0+ bytes


In [43]:
test_df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 51 to 61
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  11 non-null     object
 1   Summary  11 non-null     object
dtypes: object(2)
memory usage: 264.0+ bytes


In [44]:
test_df1

,Article,Summary
51,cnn palestinian journalist are finding reporti...,new palestinian security force say they will i...
52,washington cnn terrorist are likely to use a w...,new next terror attack on u likely to originat...
53,cavite city philippine cnn at rhandolf fajardo...,efren peaflorida wa bullied by gang in high sc...
54,by ted thornhill last updated at pm on decembe...,the free iphone app come with a missile strike...
55,by chris brooke published est april updated es...,lee rowlands tried to raise a day for charity ...
56,washington cnn the pilot of northwest flight d...,first officer didnt completely load radio freq...
57,four police officer involved in the fatal purs...,a court heard that four police officer used un...
58,by john hall scam patricia glancy conned five ...,patricia glancy conned couple out of money at ...
59,the secret to staying slim could be in saliva ...,gene make a carbbusting compound and can influ...
60,by james rush published est june updated est j...,historian seeking to identify troop of fallen ...


## **Training the model**

In [ ]:
#Defining a PyTorch dataset named "PegasusDataset" that takes tokenized inputs and corresponding labels as arguments during initialization
class PegasusDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    #This will return a single data sample from the dataset    
    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels['input_ids'][idx])  # torch.tensor(self.labels[idx])
        return item
    def __len__(self):
        return len(self.labels['input_ids'])  # len(self.labels)

In [ ]:
def CheckVals(val_texts,val_labels):
  if(val_texts is None or val_labels is None):
    return False
  return True

In [ ]:
def CreateList(value):
  return value.to_list()

In [ ]:
def pegasus(encoding, decoding):
  return PegasusDataset(encoding, decoding)

In [ ]:
from transformers import AutoConfig, AutoTokenizer

config = AutoConfig.from_pretrained('google/pegasus-large')

In [ ]:
def prepare_data(model_name, 
                 train_texts, train_labels, 
                 val_texts=None, val_labels=None, 
                 test_texts=None, test_labels=None):
  """
  Preparing input data in the format accepeted by model for fine-tuning
  """
  tokenizer = AutoTokenizer.from_pretrained('google/pegasus-large')

  #Checking if all the validation and test dataset has been provided or not
  prepare_val = CheckVals(val_texts ,val_labels )
  prepare_test = CheckVals(test_texts,test_labels)

  #Generating tokens for article and summary and passing it to pegasus function to get the formatted data
  def tokenize_data(texts, labels):
    encodings = tokenizer(CreateList(texts['Article']), truncation=True, padding=True)
    decodings = tokenizer(CreateList(labels['Summary']), truncation=True, padding=True)
    #The below function is converting the tokens of article and summary to PegasusDataset Object
    dataset_tokenized = pegasus(encodings, decodings)
    return dataset_tokenized

  train_dataset = tokenize_data(train_texts, train_labels)
  if prepare_val:
    val_dataset = tokenize_data(val_texts, val_labels)
  else:
    val_dataset=None
  if prepare_test:
    test_dataset = tokenize_data(test_texts, test_labels)
  else:
    test_dataset=None
  return train_dataset, val_dataset, test_dataset, tokenizer

In [ ]:
def torchDevice():
  if torch.cuda.is_available():
    return 'cuda'
  else:
   return 'cpu'

In [ ]:
def TrainArguments(val_dataset,output_dir):
  if val_dataset is not None:
    return TrainingArguments(
      output_dir=output_dir,           
      num_train_epochs=1,           
      per_device_train_batch_size=1,  
      per_device_eval_batch_size=1,    
      save_steps=500,                  
      save_total_limit=5,              
      evaluation_strategy='steps',     
      eval_steps=100,                  
      warmup_steps=500,                
      weight_decay=0.01,               
      logging_dir='./logs',           
      logging_steps=10,
    )
  else:
    return TrainingArguments(
      output_dir=output_dir,           
      num_train_epochs=1,           
      per_device_train_batch_size=1,  
      #Saving checkpoints after 500 steps helpful during cases of session crash
      save_steps=500,                  
      save_total_limit=5,
      #Will be considered when we increase the training step, basically it is for learning rate           
      warmup_steps=500, 
      #Regularization term used to avoid overfitting               
      weight_decay=0.01,               
      logging_dir='./logs',            
      logging_steps=10,
    )


In [ ]:
torchDevice()

'cuda'

In [ ]:
def prepare_fine_tuning(tokenizer, train_dataset, val_dataset=None, freeze_encoder=False, output_dir='./results'):
  model = PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(torchDevice())

#Keeping freeze encoder=False because fine tuning only the decoder part, also trianing data is limited so avoiding overfitting by freezing

  if val_dataset is not None:
    trainer = Trainer(
      model=PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(torchDevice()),                         
      args=TrainArguments(val_dataset,output_dir),                  
      train_dataset=train_dataset,         
      eval_dataset=val_dataset,            
      tokenizer=tokenizer
    )

  else:
    trainer = Trainer(
      model=PegasusForConditionalGeneration.from_pretrained('google/pegasus-large').to(torchDevice()),                         
      args=TrainArguments(val_dataset,output_dir),                  
      train_dataset=train_dataset,         
      tokenizer=tokenizer
    )

  return trainer

In [ ]:
#Converting train dataset from series to dataframe to ease processing
train=train_df1.to_frame()

In [ ]:
#Converting validate dataset from series to dataframe to ease processing
valid = valid_df1.to_frame()

In [45]:
test = test_df1

In [46]:
!pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## **Pegasus:**

**As I mentioned in Mandate-2, I am using pegasus as pre-trained model. Initialising, I generated the glove vectors for my dataset and decided to modify the embedded layer of the model to take glove vectors as input but it was quite challenging. Hence, giving the text as input to PTM.**

In [ ]:
train_dataset, _, _, tokenizer = prepare_data('google/pegasus-large', train, valid)

In [ ]:
trainer = prepare_fine_tuning(tokenizer, train_dataset)

In [ ]:
#Fine tuning the Pegasus on the dataset
#Function is calculating training loss for every 10 rows
#Using wandb for easily log and track metrics during training
model = trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a PegasusTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,9.544700
20,8.451700
30,9.721300
40,9.008800
50,8.243500


### **Further uploading the fine-tuned Pegasus to hugging face repository to avoid fine-tuning the model again and again.**

In [ ]:
trainer.save_model("./pegasus_model_NLP")

In [ ]:
from huggingface_hub import notebook_login

In [ ]:
notebook_login()

In [ ]:
trainer.push_to_hub("Yasha145/google_pegasus")

Cloning https://huggingface.co/Yasha145/results into local empty directory.


Upload file pytorch_model.bin:   0%|          | 1.00/2.13G [00:00<?, ?B/s]

Upload file spiece.model:   0%|          | 1.00/1.82M [00:00<?, ?B/s]

Upload file training_args.bin:   0%|          | 1.00/3.43k [00:00<?, ?B/s]

To https://huggingface.co/Yasha145/results
   6ef5320..cd743d2  main -> main

   6ef5320..cd743d2  main -> main

To https://huggingface.co/Yasha145/results
   cd743d2..3d4b675  main -> main

   cd743d2..3d4b675  main -> main



'https://huggingface.co/Yasha145/results/commit/cd743d25a51704ab95c09cdd1f130cc939cd8b4c'

## **Rogue Score Calculation**

In [26]:
pip install rouge

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [21]:
import tensorflow as tf
import transformers
from transformers import PegasusTokenizer, PegasusForConditionalGeneration


In [22]:
model_saved = PegasusForConditionalGeneration.from_pretrained("Yasha145/results");

In [23]:
from transformers import AutoConfig, AutoTokenizer

config = AutoConfig.from_pretrained('google/pegasus-large')

In [28]:
!pip install sentencepiece 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 23.2 MB/s eta 0:00:00


In [29]:
tokenizer = AutoTokenizer.from_pretrained('google/pegasus-large')

In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11 entries, 51 to 61
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Article  11 non-null     object
 1   Summary  11 non-null     object
dtypes: object(2)
memory usage: 264.0+ bytes


In [47]:
generated_summaries = []
target_summaries = []
#target_summaries = test['Summary']

In [38]:
test

,Article,Summary
51,cnn palestinian journalist are finding reporti...,new palestinian security force say they will i...
52,washington cnn terrorist are likely to use a w...,new next terror attack on u likely to originat...
53,cavite city philippine cnn at rhandolf fajardo...,efren peaflorida wa bullied by gang in high sc...
54,by ted thornhill last updated at pm on decembe...,the free iphone app come with a missile strike...
55,by chris brooke published est april updated es...,lee rowlands tried to raise a day for charity ...
56,washington cnn the pilot of northwest flight d...,first officer didnt completely load radio freq...
57,four police officer involved in the fatal purs...,a court heard that four police officer used un...
58,by john hall scam patricia glancy conned five ...,patricia glancy conned couple out of money at ...
59,the secret to staying slim could be in saliva ...,gene make a carbbusting compound and can influ...
60,by james rush published est june updated est j...,historian seeking to identify troop of fallen ...


In [35]:
type(test)

pandas.core.frame.DataFrame

In [40]:
#Converting dataframe to dictionary for ease of handling the articles
test_list = test.to_dict('records')

In [37]:
test_list

[{'Article': 'cnn palestinian journalist are finding reporting increasingly difficult in the face of the growing abuse toward them by palestinian security force in the west bank and gaza according to a report released wednesday by human right watch the report no news is good news abuse against journalist by palestinian security force criticizes palestinian authority force in the west bank and hamas security force in gaza saying they are abusing local journalist and limiting press freedom in the palestinian territory the report focus on seven incident in the west bank and two in gaza saying they are case in which security force tortured beat and arbitrarily detained journalist confiscated their equipment and barred them from leaving the west bank and gaza human right watch cite a report issued by the palestinian center for development and medium freedom that say there wa a increase in the number of physical attack arrest confiscation of equipment and limitation of movement in the year w

## **Fine-tuned model: Summary Generation against Test Dataset**

In [48]:
for example in test_list:
    inputs = tokenizer(example['Article'], truncation=True, padding='longest', return_tensors="pt")
    # inputs = tokenizer.encode(example['Article'], return_tensors='tf')
    summary_ids = model_saved.generate(**inputs)
    generated_summary=tokenizer.batch_decode(summary_ids, skip_special_tokens=True)
    generated_summaries.append(generated_summary)
    target_summaries.append(example['Summary'])

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1313: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [ ]:
type(generated_summaries)

list

In [ ]:
type(target_summaries)

list

In [49]:
!pip install rouge_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24954 sha256=79ca370c7668fff3078b293a2e3ab74e172307f4f6c5dd824dbae07d830ad78b
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [50]:
from rouge_score import rouge_scorer

In [51]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [38]:
generated_summaries

[['cnn palestinian journalist are finding reporting increasingly difficult in the face of the growing abuse toward them by palestinian security force in the west bank and gaza according to a report released wednesday by human right watch the report no news is good news abuse against journalist by palestinian security force criticizes palestinian authority force in the west bank and hamas security force in gaza saying they are abusing local journalist and limiting press freedom in the palestinian territory the report focus on seven incident in the west bank and two in gaza saying they are case in which security force tortured beat and arbitrarily detained journalist confiscated their equipment and barred them from leaving the west bank and gaza human right watch cite a report issued by the palestinian center for development and medium freedom that say there wa a increase in the number of physical attack arrest confiscation of equipment and limitation of movement in the year we respect t

In [52]:
#Converting list of list of strings to list of strings for better comparsion for score
single_list_generated_summaries = [item for sublist in generated_summaries for item in sublist]

In [ ]:
len(single_list_generated_summaries)

11

In [ ]:
len(target_summaries)

11

In [40]:
target_summaries

['new palestinian security force say they will investigate the allegation the human right watch report focus on seven incident in the west bank and two in gaza journalist were tortured beat and arbitrarily detained the report say the rift between fatah and hamas is behind most of the incident it say',
 'new next terror attack on u likely to originate in pakistan tribal area report say terrorist with biological weapon could kill million panel chairman say biological attack more likely than nuclear attack report say number of nation with nuclear weapon also growing panel say',
 'efren peaflorida wa bullied by gang in high school in the philippine now his dynamic teen company offer an alternative to gang through education since some member have taught more than child in slum vote now for the cnn hero of the year at cnncomheroes',
 'the free iphone app come with a missile strike and a car crash',
 'lee rowlands tried to raise a day for charity to win the competition but the strain got too 

## **Calculating scores against the summaries generated by the trained model and the expected summaries**

In [53]:
from rouge import Rouge
rouge = Rouge()
# Calculating scores against the summaries generated by the trained model and the expected summaries
scores = rouge.get_scores(single_list_generated_summaries, target_summaries, avg=True)
print(scores)

{'rouge-1': {'r': 0.8356025990932, 'p': 0.22437636671207664, 'f': 0.3486692866284592}, 'rouge-2': {'r': 0.4799314497535841, 'p': 0.0942670631540141, 'f': 0.15485379445029335}, 'rouge-l': {'r': 0.635093258706883, 'p': 0.17049841183233563, 'f': 0.26437905424348596}}


In [54]:
thresholds = {
    'rouge-1': 0.3,
    'rouge-2': 0.15,
    'rouge-l': 0.25
}

n_correct = 0
for i in range(len(generated_summaries)):
    is_correct = True
    for metric, threshold in thresholds.items():
        if scores[metric]['f'] < threshold:
            is_correct = False
            break
    if is_correct:
        n_correct += 1

accuracy = n_correct / len(generated_summaries)
print('Accuracy:', accuracy)

Accuracy: 1.0
